In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import re
import os,sys

import kbij_helper as kbijh
kbij = kbijh.KBIJ()

import warnings
warnings.filterwarnings('ignore')
autostats_lib = "../../../autostats"
if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)

from utilities.data_storage import DataStorage
import time

In [2]:
%load_ext google.cloud.bigquery
from google.cloud import bigquery
DataStorage(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

from IPython.display import clear_output

In [3]:
def get_key(my_dict, val):
    keys_list = []
    for key, value in my_dict.items(): 
         if val == value: 
             keys_list.append(key)
  
    return keys_list

In [4]:
date = "2020-07-01"
prev_date = "2020-06-01"

In [5]:
datem = pd.to_datetime(date)
datem = datem.strftime("%Y%m")
print(datem)

202007


In [6]:
pathname = 'data'+datem
dirname = './'+pathname

In [7]:
raw_filename_F01 = dirname+'/F01'+datem+'.csv'
raw_filename_D01 = dirname+'/D01'+datem+'.csv'

In [8]:
filtered_filename_F01 = dirname+'/F01'+datem+'_new_filtered.csv'
filtered_filename_D01 = dirname+'/D01'+datem+'_new_filtered.csv'

In [9]:
prev_datem = pd.to_datetime(prev_date)
prev_datem = prev_datem.strftime("%Y%m")
print(prev_datem)

202006


In [10]:
prev_pathname = 'data'+prev_datem
prev_dirname = './'+prev_pathname

In [11]:
prev_filtered_filename_F01 = prev_dirname+'/F01'+prev_datem+'_new_filtered.csv'
prev_filtered_filename_D01 = prev_dirname+'/D01'+prev_datem+'_new_filtered.csv'

In [12]:
F01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Kode Jenis Fasilitas': 'object',
 'Nomor Rekening Fasilitas': 'object',
 'Nomor Rekening Lama Fasilitas': 'object',
 'Nomor CIF Debitur': 'object',
 'Kode Sifat Kredit atau Pembiayaan': 'str',
 'Kode Jenis Kredit atau Pembiayaan': 'str',
 'Kode Akad Kredit atau Akad Pembiayaan': 'object',
 'Nomor Akad Awal': 'object',
 'Tanggal Akad Awal': 'object',
 'Nomor Akad Akhir': 'object',
 'Tanggal Akad Akhir': 'object',
 'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan': 'int64',
 'Tanggal Awal Kredit atau Pembiayaan': 'object',
 'Tanggal Mulai': 'object',
 'Tanggal Jatuh Tempo': 'object',
 'Kode Kategori Debitur': 'object',
 'Kode Jenis Penggunaan': 'str',
 'Kode Orientasi Penggunaan': 'str',
 'Kode Sektor Ekonomi': 'object',
 'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan': 'object',
 'Nilai Proyek': 'object',
 'Kode Valuta': 'object',
 'Suku Bunga atau Imbalan': 'float64',
 'Jenis Suku Bunga atau Imbalan': 'int64',
 'Kredit atau Pembiayaan Program Pemerintah': 'object',
 'Plafon Awal': 'float64',
 'Plafon': 'float64',
 'Realisasi atau Pencairan Bulan Berjalan': 'float64',
 'Denda': 'float64',
 'Baki Debet': 'float64',
 'Nilai dalam Mata Uang Asal': 'object',
 'Kode Kualitas Kredit atau Pembiayaan': 'str',
 'Tanggal Macet': 'object',
 'Kode Sebab Macet': 'str',
 'Tunggakan Pokok': 'int64',
 'Tunggakan Bunga atau Imbalan': 'int64',
 'Jumlah Hari Tunggakan': 'int64',
 'Frekuensi Tunggakan': 'int64',
 'Frekuensi Restrukturisasi': 'int64',
 'Tanggal Restrukturisasi Awal': 'object',
 'Tanggal Restrukturisasi Akhir': 'object',
 'Kode Cara Restrukturisasi': 'object',
 'Kode Kondisi': 'object',
 'Tanggal Kondisi': 'object',
 'Keterangan': 'object',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

D01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Nomor CIF Lama Debitur': 'object',
 'Nomor CIF Debitur': 'object',
 'Jenis Identitas': 'int64',
 'Nomor Identitas': 'object',
 'Nama Sesuai Identitas': 'object',
 'Nama Lengkap': 'object',
 'Kode Status Pendidikan atau Gelar Debitur': 'object',
 'Jenis Kelamin': 'object',
 'Tempat Lahir': 'object',
 'Tanggal Lahir': 'object',
 'Nama Gadis Ibu Kandung': 'object',
 'NPWP': 'object',
 'Alamat': 'object',
 'Kelurahan': 'object',
 'Kecamatan': 'object',
 'Kode Kabupaten atau Kota': 'object',
 'Kode Pos': 'object',
 'Nomor Telepon': 'object',
 'Nomor Telepon Seluler': 'object',
 'Alamat email': 'object',
 'Kode Negara Domisili': 'object',
 'Kode Pekerjaan': 'object',
 'Tempat Bekerja': 'object',
 'Kode Bidang Usaha Tempat Bekerja': 'object',
 'Alamat Tempat Bekerja': 'object',
 'Kode Golongan Debitur': 'object',
 'Status Perkawinan Debitur': 'int64',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

# Query

Get new F01 and D01 from redash

## F01 Current Data

In [13]:
F01new = kbij.get_F01_data('"'+date+'"')

[get_F01_data] Query from BigQuery takes 310.0338644981384


## D01 Current Data

In [14]:
D01new = kbij.get_D01_data('"'+date+'"')

[get_D01_data] Query from BigQuery takes 120.16708087921143


# Data Preparation

In [15]:
print(F01new.shape)
a=[]
for i in F01new.columns:
    a.append(i.replace('_',' '))
F01new.columns = a

(573316, 69)


In [16]:
print(D01new.shape)
a=[]
for i in D01new.columns:
    a.append(i.replace('_',' '))
D01new.columns = a

(192892, 40)


In [17]:
for col in get_key(F01_dtype, 'object'):
    F01new[col] = F01new[col].str.strip()

In [18]:
for col in get_key(D01_dtype, 'object'):
    D01new[col] = D01new[col].str.strip()

In [19]:
F01new = F01new.replace('\n','', regex=True)
D01new = D01new.replace('\n','', regex=True)

In [20]:
F01new = F01new.replace('\t','', regex=True)
D01new = D01new.replace('\t','', regex=True)

In [21]:
D01new.rename(columns={"Alamat email": "Alamat e-mail"}, inplace=True)

Check If KTP ID has char O/o

In [22]:
D01new['Nomor Identitas'][D01new['Nomor Identitas'].str.contains('O',case=False, regex=False)==True]

Series([], Name: Nomor Identitas, dtype: object)

Trim "Alamat e-mail" to 150 char, "Alamat Tempat Bekerja" to 300 char

In [23]:
D01new['Alamat e-mail']=D01new['Alamat e-mail'].str[-149:]
D01new['Alamat Tempat Bekerja']=D01new['Alamat Tempat Bekerja'].str[:299]

Check if there is still "Alamat e-mail" and "Alamat Tempat Bekerja" with more than 150 and 300 char each

In [24]:
D01new['Alamat e-mail'][D01new['Alamat e-mail'].str.len()>=150]

Series([], Name: Alamat e-mail, dtype: object)

In [25]:
D01new['Alamat Tempat Bekerja'][D01new['Alamat Tempat Bekerja'].str.len()>=300]

Series([], Name: Alamat Tempat Bekerja, dtype: object)

In [26]:
gelar_list = pd.read_csv('./data/gelar.csv')['index'].tolist()
gelar_depan_list = pd.read_csv('./data/gelar_depan.csv')['index'].tolist()

In [27]:
def remove_gelar(nama):
    nama_list = nama.split()
    clear_output()
    print(nama_list)
    if nama_list[-1].lower() in gelar_list:
        del nama_list[-1]
    if nama_list[0].lower() in gelar_depan_list:
        del nama_list[0]    
        
    return ' '.join(nama_list)

In [28]:
D01new['Nama Sesuai Identitas'] = D01new['Nama Sesuai Identitas'].apply(remove_gelar)

In [29]:
D01new['Nama Gadis Ibu Kandung'] = D01new['Nama Gadis Ibu Kandung'].apply(remove_gelar)

['WURYANTI']

In [30]:
F01new.to_csv(path_or_buf=raw_filename_F01, index=False)
D01new.to_csv(path_or_buf=raw_filename_D01, index=False)

In [31]:
DataStorage.upload_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

File ./data202007/F01202007.csv version 3 uploaded to notebooks/kbij/data202007/f01202007.csv.


In [32]:
DataStorage.upload_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

File ./data202007/D01202007.csv version 3 uploaded to notebooks/kbij/data202007/d01202007.csv.


# Load Data

In [33]:
DataStorage.download_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

Blob notebooks/kbij/data202007/f01202007.csv version LATEST downloaded to ./data202007/F01202007.csv.


In [34]:
DataStorage.download_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

Blob notebooks/kbij/data202007/d01202007.csv version LATEST downloaded to ./data202007/D01202007.csv.


In [35]:
F01new = pd.read_csv(raw_filename_F01, dtype=F01_dtype)
D01new = pd.read_csv(raw_filename_D01, dtype=D01_dtype)

In [36]:
print(prev_filtered_filename_F01)
print('kbij/'+prev_pathname)
print('f01'+prev_datem+'_new_filtered.csv')

./data202006/F01202006_new_filtered.csv
kbij/data202006
f01202006_new_filtered.csv


In [37]:
DataStorage.download_notebooks_data(prev_filtered_filename_F01, 'kbij/'+prev_pathname, 'f01'+prev_datem+'_new_filtered.csv')

Blob notebooks/kbij/data202006/f01202006_new_filtered.csv version LATEST downloaded to ./data202006/F01202006_new_filtered.csv.


In [38]:
DataStorage.download_notebooks_data(prev_filtered_filename_D01, 'kbij/'+prev_pathname, 'd01'+prev_datem+'_new_filtered.csv')

Blob notebooks/kbij/data202006/d01202006_new_filtered.csv version LATEST downloaded to ./data202006/D01202006_new_filtered.csv.


In [39]:
F01_prev = pd.read_csv(prev_filtered_filename_F01, dtype=F01_dtype)
D01_prev = pd.read_csv(prev_filtered_filename_D01, dtype=D01_dtype)

In [40]:
print('D01new:',D01new.shape, 'F01new:',F01new.shape)
print('D01_prev:',D01_prev.shape, 'F01_prev:',F01_prev.shape)

D01new: (192892, 40) F01new: (573316, 69)
D01_prev: (187594, 40) F01_prev: (549378, 69)


# Checking

In [41]:
F01new, D01new = kbij.checking_data(F01new, D01new, date, F01_prev=F01_prev, D01_prev=D01_prev)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

Found 444175 Nomor Rekening Fasilitas
--> kbij.existing_previous_not_active_data
Drop previous not active Nomor Rekening Fasilitas from current F01
Drop not existing Nomor CIF Debitur in current F01 from current D01

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

Found 114 Nomor Rekening Fasilitas
--> kbij.active_but_has_no_balance
Set Kode Kondisi to not active

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Mace

## Check column

In [42]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Frekuensi Tunggakan']>0)][['orderId','Jumlah Hari Tunggakan','Frekuensi Tunggakan','Approved Date','status']]

,orderId,Jumlah Hari Tunggakan,Frekuensi Tunggakan,Approved Date,status


In [43]:
F01new[(F01new['Tunggakan Pokok']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue


In [44]:
F01new[(F01new['Denda']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
264,ATH-11KDLXQ4,0.0,2,00,65
397,ATH-11U6MLW9,0.0,5,00,187
626,ATH-129P6LAM,0.0,4,00,146
1020,ATH-131VGPK9,0.0,4,00,158
1388,ATH-13SVCYFQ,0.0,5,00,482
1488,ATH-13Z27H2L,0.0,4,00,140
1909,ATH-14T7HLXJ,0.0,5,00,381
2308,ATH-15KDJSLQ,0.0,5,00,234
2322,ATH-15LJUAP8,0.0,5,00,247
2369,ATH-15PRH8ZH,0.0,5,00,576


In [45]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Tunggakan Bunga atau Imbalan','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Tunggakan Bunga atau Imbalan,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
78198,ATH-5LEAQ3CS,0,661000.0,5,00,593
100783,ATH-6XQVT373,0,950000.0,5,00,270
103638,ATH-74BYYA6K,0,950000.0,5,00,216
112585,ATH-7MWFASZQ,0,420000.0,2,00,64
268865,ATH-GTBSMBMB,0,458000.0,5,00,403
291598,ATH-J56C6R33,0,123000.0,3,00,112
355903,ATH-MW7D4MYP,0,950000.0,5,00,771
467009,ATH-UCQBBQMV,0,661000.0,5,00,501
505945,ATH-WMZPNWPX,0,661000.0,5,00,505
563647,EXT-11D5L6KH,0,950000.0,2,00,34


In [46]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [47]:
F01new[(F01new['Frekuensi Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [48]:
F01new[(F01new['Denda']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [49]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [50]:
F01new[(F01new['Kode Sebab Macet']=='99') & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1')][['orderId','Kode Sebab Macet','Kode Kualitas Kredit atau Pembiayaan']]

,orderId,Kode Sebab Macet,Kode Kualitas Kredit atau Pembiayaan


In [51]:
F01new[(~F01new['Tanggal Macet'].isnull()) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1')][['orderId','Tanggal Macet','Kode Kualitas Kredit atau Pembiayaan']]

,orderId,Tanggal Macet,Kode Kualitas Kredit atau Pembiayaan


# Final File to Send to KBIJ D01 and F01

Null check for CIF and NIK

In [52]:
D01new.loc[(D01new['Nomor CIF Debitur'].isna())|(D01new['Nama Lengkap'].isna())|(D01new['Tanggal Lahir'].isna())|(D01new['Nomor Identitas'].isna())]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [53]:
D01new.drop(D01new.loc[(D01new['Nomor CIF Debitur'].isna())|(D01new['Nama Lengkap'].isna())|(D01new['Tanggal Lahir'].isna())|(D01new['Nomor Identitas'].isna())].index, inplace=True)

In [54]:
F01new.loc[F01new['Nomor CIF Debitur'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,lateFee,balanceWithInterest,balance,dataDate,jumlah cicilan,current payment periode month,count payment,paidoffDate,status,Tunggakan Pokok


In [55]:
F01new.drop(F01new.loc[F01new['Nomor CIF Debitur'].isna()].index, inplace=True)

## Export full data dump to csv to be used in the next iteration

In [56]:
F01new.to_csv(path_or_buf=filtered_filename_F01, index=0)
D01new.to_csv(path_or_buf=filtered_filename_D01, index=0)

In [57]:
DataStorage.upload_notebooks_data(filtered_filename_F01, 'kbij/'+pathname, 'f01'+datem+'_new_filtered.csv')

File ./data202007/F01202007_new_filtered.csv version 4 uploaded to notebooks/kbij/data202007/f01202007_new_filtered.csv.


In [58]:
DataStorage.upload_notebooks_data(filtered_filename_D01, 'kbij/'+pathname, 'd01'+datem+'_new_filtered.csv')

File ./data202007/D01202007_new_filtered.csv version 4 uploaded to notebooks/kbij/data202007/d01202007_new_filtered.csv.


Export Latest Data

In [59]:
timestr = time.strftime("%Y%m%d-%H%M%S")
F01name = dirname+'/F01'+'-'+datem+'['+timestr+']'+'.txt'
D01name = dirname+'/D01'+'-'+datem+'['+timestr+']'+'.txt'
print(F01name,D01name)

./data202007/F01-202007[20200921-133034].txt ./data202007/D01-202007[20200921-133034].txt


In [60]:
F01new_sent=F01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
       'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
       'Kode Sifat Kredit atau Pembiayaan',
       'Kode Jenis Kredit atau Pembiayaan',
       'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
       'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
       'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
       'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
       'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
       'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
       'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
       'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
       'Jenis Suku Bunga atau Imbalan',
       'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
       'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
       'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
       'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
       'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
       'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
       'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
       'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
       'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
       'Create Date', 'Update Date']]
D01new_sent=D01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]

In [61]:
D01new[D01new['Nama Sesuai Identitas']=='IBNU SIGIT SAPUTRO']

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [62]:
print(F01new_sent.shape)
print(D01new_sent.shape)

(129141, 53)
(120579, 35)


## Export file for KBIJ Reporting

In [63]:
D01new_sent.to_csv(path_or_buf=D01name, sep='|',index =0)
F01new_sent.to_csv(path_or_buf=F01name, sep='|',index =0)

In [64]:
DataStorage.upload_notebooks_data(F01name, 'kbij/'+pathname, 'f01-'+datem+'.txt')

File ./data202007/F01-202007[20200921-133034].txt version 3 uploaded to notebooks/kbij/data202007/f01-202007.txt.


In [65]:
DataStorage.upload_notebooks_data(D01name, 'kbij/'+pathname, 'd01-'+datem+'.txt')

File ./data202007/D01-202007[20200921-133034].txt version 3 uploaded to notebooks/kbij/data202007/d01-202007.txt.


# Check Sent Data

In [66]:
F01_sent = pd.read_csv('./data202007/F01-202007[20200921-133034].txt',sep='|',dtype=F01_dtype)
D01_sent = pd.read_csv('./data202007/D01-202007[20200921-133034].txt',sep='|',dtype=D01_dtype)

In [67]:
F01_sent[F01_sent['Tanggal Mulai'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


In [68]:
F01_sent[F01_sent['Tanggal Jatuh Tempo'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


In [69]:
DataStorage.download_notebooks_data('./'+prev_pathname+'/f01-'+prev_datem+'.txt', 'kbij/'+prev_pathname, 'f01-'+prev_datem+'.txt')

Blob notebooks/kbij/data202006/f01-202006.txt version LATEST downloaded to ./data202006/f01-202006.txt.


In [70]:
DataStorage.download_notebooks_data('./'+prev_pathname+'/d01-'+prev_datem+'.txt', 'kbij/'+prev_pathname, 'd01-'+prev_datem+'.txt')

Blob notebooks/kbij/data202006/d01-202006.txt version LATEST downloaded to ./data202006/d01-202006.txt.


In [71]:
F01_prev_sent = pd.read_csv('./'+prev_pathname+'/f01-'+prev_datem+'.txt',sep='|',dtype=F01_dtype)
D01_prev_sent = pd.read_csv('./'+prev_pathname+'/d01-'+prev_datem+'.txt',sep='|',dtype=D01_dtype)

In [72]:
print('D01_sent:',D01_sent.shape, 'F01_sent:',F01_sent.shape)
print('D01_prev_sent:',D01_prev_sent.shape, 'F01_prev_sent:',F01_prev_sent.shape)

D01_sent: (120579, 35) F01_sent: (129141, 53)
D01_prev_sent: (187594, 35) F01_prev_sent: (549378, 53)


In [73]:
F01_sent, D01_sent = kbij.checking_data(F01_sent, D01_sent, '2020-05-01', F01_prev=F01_prev_sent, D01_prev=D01_prev_sent)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

[CLEAR]No Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

[CLEAR]No Active Kode Kondisi but Baki Debet = 0

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet 

[CLEAR]No Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet

[DONE]
Check Tahun Bulan Data 

F01: ['202007']
D01: ['202007']

[DONE]


In [74]:
D01_sent['Alamat e-mail'][D01_sent['Alamat e-mail'].str.len()>=150]

Series([], Name: Alamat e-mail, dtype: object)

In [75]:
D01_sent['Alamat Tempat Bekerja'][D01_sent['Alamat Tempat Bekerja'].str.len()>=300]

Series([], Name: Alamat Tempat Bekerja, dtype: object)